In [3]:
%pip install torch --index-url https://download.pytorch.org/whl/cu130

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu130
  Using cached https://download.pytorch.org/whl/cu130/torch-2.9.1%2Bcu130-cp313-cp313-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/1.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 GB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 GB 4.0 MB/s eta 0:07:41
   ---------------------------------------- 0.0/1.9 GB 3.8 MB/s eta 0:08:09
   ---------------------------------------- 0.0/1.9 GB 3.5 MB/s eta 0:08:54
   ---------------------------------------- 0.0/1.9 GB 3.6 MB/s eta 0:08:41
   -----------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\joanc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached xgboost-3.1.2-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached pyyaml-6.0.3-cp313-cp313-win_amd64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp313-cp313-win_amd64.whl.metadata (41 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.7.0-cp38-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl (11.0 MB)
Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl (12.8 MB)
   ---------------------------------------- 0.0/72.0 MB ? eta -:--:--
   ------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.5 which is incompatible.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\joanc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Importar dependencias
import xgboost as xgb
from src.v2.data_loader import data_generator
from sklearn.model_selection import train_test_split
from src.v2.maps import OTHER_LABEL

C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# Paths de los archivos
csv_file = "../../data/processed/dataset_reducido_30p.csv"
model_output = "../../models/modeloxgb_vulnerabilidades.json"

In [26]:
# Inicializar variables para evaluación final
X_test, y_test = [], []

In [ ]:
# Función principal de entrenamiento incremental
def train_incremental():

    # Parámetros de XGBoost
    params = {
        ## Probabilidades multiclase
        'objective': 'multi:softprob',
        ## Número de clases (0=Safe, 1..N=Top CWEs, N+1=Other)  
        'num_class': OTHER_LABEL + 1,
        ## Profundidad máxima de cada árbol
        'max_depth': 10,
        ## Tasa de aprendizaje (eta) 
        'learning_rate': 0.05,
        ## Método de construcción de árboles (hist: basado en histogramas)
        'tree_method': 'hist',
        ## Uso de GPU para entrenamiento
        'device': 'cuda',
        ## Métrica de evaluación (logaritmo de pérdida para clasificación multiclase)
        'eval_metric': 'mlogloss',
        ## Uso de todos los núcleos posibles
        'n_jobs': -1,
        ## Muestreo de columnas: Cada vez que crea un árbol, usa solo el 80% de las columnas al azar.
        'colsample_bytree': 0.8, 
        ## Muestreo de filas: Cada árbol se entrena con un subconjunto aleatorio del 80% de los datos.
        'subsample': 0.8,
        ## Peso mínimo en hojas valores comunes [1, 3, 5]
        'min_child_weight': 3
    }
    
    model = None
    chunk_idx = 0
    
    # Iteramos sobre el generador
    for X_chunk, y_chunk in data_generator(csv_file, chunk_size=5000):
        chunk_idx += 1
        print(f"-------- Entrenando Chunk #{chunk_idx} (Tamaño: {len(y_chunk)}) --------")
        
        # Opcional: Hacer un split interno para guardar las particiones para evaluar al final
        X_train, X_val, y_train, y_val = train_test_split(X_chunk, y_chunk, test_size=0.1)
        X_test.append(X_val)
        y_test.append(y_val)
        
        # Convertir a DMatrix (formato optimizado de XGBoost)
        dtrain = xgb.DMatrix(X_train, label=y_train)
        
        # ENTRENAMIENTO INCREMENTAL
        # xgb_model=model le dice que continúe desde el estado anterior
        # num_boost_round= añade N árboles nuevos por cada chunk
        model = xgb.train(
            params, 
            dtrain, 
            num_boost_round=10, 
            xgb_model=model
        )

        # Guardar checkpoint cada cierto tiempo
        if chunk_idx % 5 == 0:
            model.save_model(f"checkpoint_{chunk_idx}.json")
            print(f"[INFO] Checkpoint guardado: checkpoint_{chunk_idx}.json")

    # Guardar modelo final
    model.save_model(model_output)
    print(f"[INFO] Entrenamiento finalizado. Modelo guardado en {model_output}")

In [ ]:
## Entrenar el modelo
train_incremental()

--- CodeBERT cargado en: cuda ---
[INFO] Iniciando lectura de ../../data/processed/dataset_reducido_30p.csv en chunks de 5000...
-------- Entrenando Chunk #1 (Tamaño: 5000) --------
-------- Entrenando Chunk #2 (Tamaño: 5000) --------
-------- Entrenando Chunk #3 (Tamaño: 5000) --------


In [15]:
# Importar librerías de metricas
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [19]:
## Evaluar el modelo entrenado
model = xgb.Booster()
model.load_model(model_output)

X_test_full = np.vstack(X_test)
y_test_full = np.concatenate(y_test)

print(f"[INFO] Tamaño del conjunto de test: {X_test_full.shape}")

dtest = xgb.DMatrix(X_test_full, label=y_test_full)
preds = model.predict(dtest)

best_preds = [int(np.argmax(line)) for line in preds]
print(classification_report(y_test_full, best_preds))

# Matriz de confusión
cm = confusion_matrix(y_test_full, best_preds)
print("\n Matriz de Confusión:")
print(cm)


[INFO] Tamaño del conjunto de test: (1813, 804)
              precision    recall  f1-score   support

           0       0.67      0.96      0.79      1068
           1       0.62      0.07      0.12       144
           2       0.00      0.00      0.00         6
           3       1.00      0.09      0.16        90
           4       0.80      0.20      0.32        20
           5       0.62      0.10      0.18        48
           6       0.00      0.00      0.00        11
           7       1.00      0.38      0.55        89
           8       0.00      0.00      0.00         2
          10       0.00      0.00      0.00        15
          11       0.53      0.34      0.42       320

    accuracy                           0.66      1813
   macro avg       0.48      0.20      0.23      1813
weighted avg       0.66      0.66      0.59      1813


 Matriz de Confusión:
[[1022    6    0    0    0    3    0    0    0    0   37]
 [ 124   10    0    0    0    0    0    0    0    0   10]


C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\joanc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\sklearn\metrics\_clas